# Q Function
* used to approximate the reward based on a state
* Q(s,a) calculates the expected future value from state **s** and action **a**
* in DQN, we use a **neural network to approximate the reward**

# Classes
* Environment
* Agent
* Runner

## Environment

In [1]:
class Action:
    def __init__(self, act, days, value):
        self.act = act
        self.days = days
        self.value = value

In [2]:
import numpy as np
import pandas_datareader as pdr
import datetime

BUY = 'buy'
SELL = 'sell'
SKIP = 'skip'

class Environment:
    
    max_days_to_hold = 5
    min_days_to_hold = 1
    default_stock_value = 100
    
    def __init__(self, 
                 ticker, 
                 initial_deposit = 100000,
                 from_date = datetime.datetime(2007, 1, 1), 
                 to_date = datetime.datetime(2017, 1, 1),
                 window = 30):
        self.initial_deposit = initial_deposit
        self.window = window
        self.data = pdr.get_data_google(ticker, from_date, to_date)
        self.data_length = len(self.data)
        
        self.min_date = self.data.index.min()
        self.max_date = self.data.index.max()
        
        actions = np.array([BUY, SELL, SKIP])
        days_to_holds = np.arange(Environment.min_days_to_hold, 
                                  Environment.max_days_to_hold + 1,
                                  1)
        
        self.action_space = [Action(act, days, Environment.default_stock_value) for act in actions for days in days_to_holds]
        self.reset()
        
    def reset(self):
        self.deposit = self.initial_deposit
        self.current_index = self.window
        return self.state()
    
    def score(self):
        return self.deposit
    
    def enough_data_provided(self):
        return self.current_index + Environment.max_days_to_hold <= self.data_length
    
    def _current_price(self):
        return self.data.iloc[self.current_index]['Close']
    
    def state(self):
        return self.data.iloc[self.current_index - self.window:self.current_index]['Close']
        
    def state_size(self):
        return self.window
    
    def action_size(self):
        return len(self.action_space)
        
    def step(self, action_idx: int):
        action = self.action_space[action_idx]
        #print('\t=> current action is: {} at {}'.format(action, self.data.index[self.current_index]))
        
        df = self.data.iloc[self.current_index: self.current_index + action.days]['Close']
        first_day_price = df.iloc[0]
        last_day_price = df.iloc[-1]
        
        if action.act == BUY:
            reward = last_day_price - first_day_price
        elif action.act == SELL:
            reward = first_day_price - last_day_price
        elif action.act == SKIP:
            reward = 0
        
        self.current_index += action.days
        self.deposit += reward * action.value
        
        next_state = self.state()
        done = False
        _ = None
        return next_state, reward, done, _ 

## Agent

In [3]:
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.losses import mean_squared_error

class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
    
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=mean_squared_error,
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

Using TensorFlow backend.


## Runner

In [4]:
env = Environment('AAPL')
state_size = env.state_size()
action_size = env.action_size()
print('Action size: {}, state size: {}'.format(action_size, state_size))

Action size: 15, state size: 30


In [ ]:
agent = Agent(state_size, action_size)
EPISODES = 1000
batch_size = 32

In [ ]:
for e in range(EPISODES):
    state = env.reset()
    state = state.values.reshape([1, state_size])
    while env.enough_data_provided():
        action_idx = agent.act(state)
        next_state, reward, done, _ = env.step(action_idx) # build these parameters into the NN model
        # reward = reward if not done else -10
        next_state = next_state.values.reshape([1, state_size])
        agent.remember(state, action_idx, reward, next_state, done)
        state = next_state
        
    agent.update_target_model()
    print("episode: {}/{}, score: {}, e: {:.2}".format(e, EPISODES, env.score(), agent.epsilon))
    agent.replay(batch_size)

episode: 0/1000, score: 92485.0, e: 1.0
episode: 1/1000, score: 97029.0, e: 0.99
episode: 2/1000, score: 99913.0, e: 0.98
episode: 3/1000, score: 101601.0, e: 0.97
episode: 4/1000, score: 99579.0, e: 0.96
episode: 5/1000, score: 100958.0, e: 0.95
episode: 6/1000, score: 100620.0, e: 0.94
episode: 7/1000, score: 99242.0, e: 0.93
episode: 8/1000, score: 101291.0, e: 0.92
episode: 9/1000, score: 100253.0, e: 0.91
episode: 10/1000, score: 101010.0, e: 0.9
episode: 11/1000, score: 99957.0, e: 0.9
episode: 12/1000, score: 102655.0, e: 0.89
episode: 13/1000, score: 99996.0, e: 0.88
episode: 14/1000, score: 96257.0, e: 0.87
episode: 15/1000, score: 101659.0, e: 0.86
episode: 16/1000, score: 104321.0, e: 0.85
episode: 17/1000, score: 111152.0, e: 0.84
episode: 18/1000, score: 94870.0, e: 0.83
episode: 19/1000, score: 101042.0, e: 0.83
episode: 20/1000, score: 95964.0, e: 0.82
episode: 21/1000, score: 100259.0, e: 0.81
episode: 22/1000, score: 99323.0, e: 0.8
episode: 23/1000, score: 96534.0, e:

episode: 192/1000, score: 104212.0, e: 0.15
episode: 193/1000, score: 94878.0, e: 0.14
episode: 194/1000, score: 101043.0, e: 0.14
episode: 195/1000, score: 90355.0, e: 0.14
episode: 196/1000, score: 98985.0, e: 0.14
episode: 197/1000, score: 100794.0, e: 0.14
episode: 198/1000, score: 99147.0, e: 0.14
episode: 199/1000, score: 100426.0, e: 0.14
episode: 200/1000, score: 101939.0, e: 0.13
episode: 201/1000, score: 101206.0, e: 0.13
episode: 202/1000, score: 102738.0, e: 0.13
episode: 203/1000, score: 103897.0, e: 0.13
episode: 204/1000, score: 99983.0, e: 0.13
episode: 205/1000, score: 95469.0, e: 0.13
episode: 206/1000, score: 99832.0, e: 0.13
episode: 207/1000, score: 103984.0, e: 0.12
episode: 208/1000, score: 100227.0, e: 0.12
episode: 209/1000, score: 105269.0, e: 0.12
episode: 210/1000, score: 107890.0, e: 0.12
episode: 211/1000, score: 101411.0, e: 0.12
episode: 212/1000, score: 109753.0, e: 0.12
episode: 213/1000, score: 100649.0, e: 0.12
episode: 214/1000, score: 102163.0, e: 